In [265]:
%matplotlib inline

import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.miscmodels.ordinal_model import OrderedModel
import networkx as nx

pd.options.display.max_seq_items = 2000

# Two crs, one for coordinates and a projected one to compute areas
crs_geographical = "epsg:4326"
crs_projected = "epsg:28992"

# Confounder names
confounders = [ 
           "P_00_14_JR", 'P_15_24_JR','P_25_44_JR','P_45_64_JR', 'P_65_EO_JR',  # Age distribution
           'P_LAAGINKP', 'BEV_DICHTH', 'WWB_UITTOT', 'WW_UIT_TOT', 'AO_UIT_TOT' # Confounders
           ]

# Small number to add 
epsilon = 1e-8

In [266]:
# Read data
gdf = gpd.read_file("data/prepared_data.csv", 
                    GEOM_POSSIBLE_NAMES="geometry", # Having a column named 'geometry' is too hard for geopandas
                    KEEP_GEOM_COLUMNS="NO")

# Don't ask me where this column came from
gdf.drop(["field_1"], axis=1, inplace=True)

# Set the columns as floats 
for col in confounders + ["percent_green", "percent_blue", "DEPRESSION_RISK"]:
    gdf[col] = gdf[col].astype('float')

    # Normalize the column 
    gdf[col] = (gdf[col] - gdf[col].min()) / (gdf[col].max() - gdf[col].min())
    # print(gdf[col].min(), gdf[col].mean(), gdf[col].max() )

print(gdf["DEPRESSION_RISK"].min(), gdf["DEPRESSION_RISK"].mean(), gdf["DEPRESSION_RISK"].max() )

0.0 0.528108629721533 1.0


In [277]:
class PropensityMatching():
    def __init__(self,
                 name,
                 y,
                 z, 
                 conf,
                 null = False
        ) -> None:

        self.name = name
        self.y = y.values
        self.z = z.values
        if null:
            np.random.shuffle(self.z)
        self.conf = conf
        

    # TODO: Maybe make more bins that are uneven
    # TODO: More bins when we get more data? 
    def CalculatePropensityScore(self, nbins, plot=False):
        edges = np.array([-0.000001] + list(np.linspace(0.00000001, self.z.max()+epsilon, nbins)))

        self.discrete= pd.cut(self.z, bins=edges, labels=np.arange(0,nbins))
        
        if plot:
            # TODO: Make this look nice
            plt.hist(self.discrete, bins=nbins)
            plt.show()
        
        mod_log = OrderedModel(self.z, self.conf, distr='logit')

        res_log = mod_log.fit(method='bfgs', disp=False)

        predicted = res_log.model.predict(res_log.params, exog=self.conf)
        print(type(predicted))

        self.propensity = np.array([predicted[i, self.discrete[i]] for i in range(len(self.z))])
        return self.propensity


    def DistanceMatrix(self):
        self.d = np.zeros(shape=(len(self.z), len(self.z)))

        for i in range(self.d.shape[0]):
            for j in range(self.d.shape[1]):
                self.d[i,j] = (abs(self.propensity[i] - self.propensity[j])) / (abs(self.z[i] - self.z[j])+epsilon)
        return self.d


    def MinWeightMaximalMatch(self):
        G = nx.Graph(self.d)
        match_unordered = nx.min_weight_matching(G)

        # array with low in col 0 and high in col 1
        self.match = np.zeros(shape=(len(match_unordered), 2), dtype=int)

        for idx, pair in enumerate(match_unordered):
            i = pair[0]
            j = pair[1]
            if self.z[i] < self.z[j]:
                self.match[idx,0] = i
                self.match[idx,1] = j
            else:
                self.match[idx,0] = j
                self.match[idx,1] = i
        return self.match


    def TreatmentEffect(self, threshold=None):
        self.TE = []
        for i, j in self.match:
            te = (self.y[i] - self.y[j])/(self.z[i] - self.z[j])
            if not threshold or abs(te) < threshold:
                self.TE.append(te)
        self.TE = np.array(self.TE)
        self.ATE = np.sum(self.TE)
        return self.TE, self.ATE
    
    
    def HighAndLowGroups(self, dec=4):
        high = []
        low = []
        for i, j in self.match:
            low.append(self.z[i])
            high.append(self.z[j])

        print(f"Treatment distribution over high and low groups")
        print(f"High: mean={round(np.mean(high),dec)}, std={round(np.std(high),dec)}, min={round(np.min(high),dec)}, max={round(np.max(high),dec)}")
        print(f"Low: mean={round(np.mean(low),dec)}, std={round(np.std(low),dec)}, min={round(np.min(low),dec)}, max={round(np.max(low),dec)}")
    
    def DescribeTreatmentEffect(self, dec=3):
        print(
            self.name, 
            round(self.ATE, dec),
            round(self.TE.min(), dec),
            round(np.percentile(self.TE, 25), dec),
            round(np.percentile(self.TE, 75), dec),
            round(self.TE.max(), dec)
        )


In [268]:
# Run the process for greenspace
green = PropensityMatching(
    "Green",
    gdf["DEPRESSION_RISK"], 
    gdf['percent_green'],
    gdf[confounders+["percent_blue"]]
)
green.CalculatePropensityScore(10, False)
green.DistanceMatrix()
green.MinWeightMaximalMatch()
green.TreatmentEffect()
green.HighAndLowGroups()
green.ATE

/home/levi/.local/lib/python3.10/site-packages/statsmodels/miscmodels/ordinal_model.py:419: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  xb = xb[:, None]


<class 'numpy.ndarray'>
Treatment distribution over high and low groups
High: mean=0.1226, std=0.1881, min=0.0002, max=1.0
Low: mean=0.0, std=0.0, min=0.0, max=0.0


3879.808952525429

In [269]:
# Run the process for bluespace
blue = PropensityMatching(
    "Blue",
    gdf["DEPRESSION_RISK"], 
    gdf['percent_blue'],
    gdf[confounders+["percent_green"]]
)
blue.CalculatePropensityScore(10, False)
blue.DistanceMatrix()
blue.MinWeightMaximalMatch()
blue.TreatmentEffect()
blue.HighAndLowGroups()
blue.ATE

/home/levi/.local/lib/python3.10/site-packages/statsmodels/miscmodels/ordinal_model.py:419: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  xb = xb[:, None]


<class 'numpy.ndarray'>
Treatment distribution over high and low groups
High: mean=0.2158, std=0.2076, min=0.0281, max=1.0
Low: mean=0.0524, std=0.0409, min=0.0, max=0.2044


-12.220740930671901

In [270]:
# Run the process for green- and bluespace
greenblue = PropensityMatching(
    "Green and Blue",
    gdf["DEPRESSION_RISK"], 
    gdf['percent_blue']+gdf["percent_green"],
    gdf[confounders]
)
greenblue.CalculatePropensityScore(10, False)
greenblue.DistanceMatrix()
greenblue.MinWeightMaximalMatch()
greenblue.TreatmentEffect()
greenblue.HighAndLowGroups()
greenblue.ATE

/home/levi/.local/lib/python3.10/site-packages/statsmodels/miscmodels/ordinal_model.py:419: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  xb = xb[:, None]


<class 'numpy.ndarray'>
Treatment distribution over high and low groups
High: mean=0.3132, std=0.2629, min=0.0266, max=1.1779
Low: mean=0.0777, std=0.0739, min=0.0, max=0.6278


578.9067703347102

In [273]:
# Run the null model for greenspace
greennull = PropensityMatching(
    "Green, null",
    gdf["DEPRESSION_RISK"], 
    gdf['percent_green'],
    gdf[confounders+["percent_blue"]],
    null = True
)
greennull.CalculatePropensityScore(10, False)
greennull.DistanceMatrix()
greennull.MinWeightMaximalMatch()
greennull.TreatmentEffect()
greennull.HighAndLowGroups()
greennull.ATE

/home/levi/.local/lib/python3.10/site-packages/statsmodels/miscmodels/ordinal_model.py:419: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  xb = xb[:, None]


<class 'numpy.ndarray'>
Treatment distribution over high and low groups
High: mean=0.1226, std=0.1881, min=0.0, max=1.0
Low: mean=0.0, std=0.0, min=0.0, max=0.0005


/tmp/ipykernel_6877/1534793314.py:72: RuntimeWarning: divide by zero encountered in double_scalars
  te = (self.y[i] - self.y[j])/(self.z[i] - self.z[j])


-inf

In [274]:
# Run the null model for bluespace
bluenull = PropensityMatching(
    "Blue, null",
    gdf["DEPRESSION_RISK"], 
    gdf['percent_blue'],
    gdf[confounders+["percent_green"]],
    null = True
)
bluenull.CalculatePropensityScore(10, False)
bluenull.DistanceMatrix()
bluenull.MinWeightMaximalMatch()
bluenull.TreatmentEffect()
bluenull.HighAndLowGroups()
bluenull.ATE

/home/levi/.local/lib/python3.10/site-packages/statsmodels/miscmodels/ordinal_model.py:419: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  xb = xb[:, None]


<class 'numpy.ndarray'>
Treatment distribution over high and low groups
High: mean=0.2197, std=0.2056, min=0.0303, max=1.0
Low: mean=0.0485, std=0.034, min=0.0, max=0.1612


102.79324463728683

In [275]:
# Run the null model for green- and bluespace
greenbluenull = PropensityMatching(
    "Green and Blue, null",
    gdf["DEPRESSION_RISK"], 
    gdf['percent_blue']+gdf["percent_green"],
    gdf[confounders],
    null = True
)
greenbluenull.CalculatePropensityScore(10, False)
greenbluenull.DistanceMatrix()
greenbluenull.MinWeightMaximalMatch()
greenbluenull.TreatmentEffect()
greenbluenull.HighAndLowGroups()
greenbluenull.ATE

/home/levi/.local/lib/python3.10/site-packages/statsmodels/miscmodels/ordinal_model.py:419: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  xb = xb[:, None]


<class 'numpy.ndarray'>
Treatment distribution over high and low groups
High: mean=0.3185, std=0.2586, min=0.0436, max=1.225
Low: mean=0.0723, std=0.0528, min=0.0, max=0.2316


-26.84802568638616

In [278]:
print("Name", "ATE", "Min", "25th%", "75th%", "Max")
green.DescribeTreatmentEffect()
blue.DescribeTreatmentEffect()
greenblue.DescribeTreatmentEffect()
greennull.DescribeTreatmentEffect()
bluenull.DescribeTreatmentEffect()
greenbluenull.DescribeTreatmentEffect()


Name ATE Min 25th% 75th% Max
Green 3879.808952525429 -350.35318542605995 -4.214504769401132 1.8040713203319458 2103.3031587583177
Blue -12.220740930671901 -38.42281862122904 -1.4723924723269775 1.5469496506319957 41.9552685342247
Green and Blue 578.9067703347102 -14.8050384376871 -0.5124055585884314 0.8897614885526016 292.3420235288157
Green, null -inf -inf -3.8888831315919012 2.204309161917607 669.3278039391315
Blue, null 102.79324463728683 -29.498088715363313 -1.0006310927059616 1.511597276480157 56.0541130041001
Green and Blue, null -26.84802568638616 -46.31798758569894 -0.9651424334048957 1.0861855100337166 28.43657577293679
